In [1]:
from utils import *

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


In [2]:
focus_logits, focus_cache = get_focus_logits_and_cache()

In [3]:
from descision_trees import DecisionTree
from descision_trees import train_and_evaluate_decision_tree
from create_dataset import get_big_dataset
from create_dataset import get_filtered_dataset
from descision_trees import evaluate_decision_tree
from descision_trees import get_data

In [52]:
layer = 1
neuron = 2
decision_tree, weighted_f1, f1, variable_count, rules = train_and_evaluate_decision_tree(layer, neuron, min_impurity_decrease=0.000005)

In [53]:
weighted_f1, f1

(0.7851149278867686, 0.5815602836879432)

In [54]:
rules

['(G4 not yours AND G4 not yours AND G4 not yours => -0.0135)',
 '(G4 not yours AND G4 not yours AND G4 yours AND G4 not flipped AND H6 not yours => -0.0349)',
 '(G4 not yours AND G4 not yours AND G4 yours AND G4 not flipped AND H6 yours AND C2 not mine => -0.1096)',
 '(G4 not yours AND G4 not yours AND G4 yours AND G4 not flipped AND H6 yours AND C2 mine => -0.0526)',
 '(G4 not yours AND G4 not yours AND G4 yours AND G4 flipped => -0.0708)',
 '(G4 not yours AND G4 yours AND G4 not yours AND B4 not placed AND D7 not placed AND G4 not yours AND F5 not mine => -0.0438)',
 '(G4 not yours AND G4 yours AND G4 not yours AND B4 not placed AND D7 not placed AND G4 not yours AND F5 mine AND G4 not flipped AND G3 not empty AND H5 not yours => -0.0817)',
 '(G4 not yours AND G4 yours AND G4 not yours AND B4 not placed AND D7 not placed AND G4 not yours AND F5 mine AND G4 not flipped AND G3 not empty AND H5 yours => -0.1133)',
 '(G4 not yours AND G4 yours AND G4 not yours AND B4 not placed AND D7 n

In [55]:
focus_dataset = get_big_dataset(layer, num_samples=1000, inference_size=1000)
dataset = get_filtered_dataset(focus_dataset, layer, neuron, size = 1000, overfitting_strength=None)

In [56]:
dataset_columns_new = dataset.columns.tolist()

dataset_columns_new = [col_name for col_name in dataset_columns_new if col_name[3:6] != "not"]
dataset = dataset[dataset_columns_new]

column_names = dataset.columns
column_names_input = column_names[:-1]
column_names_output = [column_names[-1]]

X = dataset[column_names_input].astype(float)
y = dataset[column_names_output].astype(float)
# turn into numpy arrays
X = X.to_numpy()
y = y.to_numpy()

In [57]:
# X_train, X_test, y_train, y_test, column_names_input = get_data(layer, neuron, focus_dataset, size=200, overfitting_strength=None)
decision_tree.column_names_input = column_names_input
weighted_f1, f1, variable_count, rules, y_pred = evaluate_decision_tree(decision_tree, X, y)

In [58]:
thresh = 0.9
dataset[(dataset["G4 yours"] > thresh) & (dataset["H6 placed"] > thresh)]
# 838

,A0 empty,A0 yours,A0 mine,A1 empty,A1 yours,A1 mine,A2 empty,A2 yours,A2 mine,A3 empty,...,G7 placed,H0 placed,H1 placed,H2 placed,H3 placed,H4 placed,H5 placed,H6 placed,H7 placed,neuron activation
168,9.999942e-01,1.278129e-06,4.539042e-06,1.207088e-09,3.503784e-11,1.000000e+00,9.999968e-01,8.595253e-10,3.275740e-06,9.999989e-01,...,8.662207e-08,3.142968e-09,2.877788e-08,3.043133e-09,1.613756e-09,8.297376e-09,2.344918e-09,1.0,1.427496e-09,0.376716
446,9.999962e-01,7.456903e-07,3.050032e-06,4.629165e-09,1.892023e-06,9.999981e-01,3.159814e-07,9.999182e-01,8.150104e-05,9.999999e-01,...,4.716684e-10,9.408218e-10,3.659841e-09,2.834457e-09,6.063925e-10,4.601339e-09,2.498139e-08,1.0,2.696478e-09,-0.126884
571,1.478976e-08,1.293545e-05,9.999870e-01,4.483378e-08,2.357783e-05,9.999764e-01,3.620681e-11,6.092867e-04,9.993907e-01,2.958454e-09,...,5.055180e-10,1.181157e-08,3.146661e-10,2.564824e-09,3.883802e-09,1.434002e-09,3.661505e-08,1.0,3.847600e-09,-0.158332
763,1.000000e+00,1.514412e-08,3.245498e-08,9.999999e-01,2.547613e-09,6.431524e-08,1.000000e+00,1.551034e-10,4.172690e-10,1.000000e+00,...,2.435499e-12,1.155020e-11,2.641808e-12,1.059807e-11,1.350425e-10,3.825449e-13,6.674063e-13,1.0,5.800030e-13,-0.169505


In [59]:
for i in range(len(y_pred)):
    if y_pred[i] > 1 and y[i] < 0:
        print(y_pred[i], y[i], i)

1.12076145 [-0.07175618] 116
1.6634213 [-0.11778547] 332
1.4542689 [-0.13358971] 488
1.1486917 [-0.10343774] 643
1.0185534 [-0.12514618] 852


In [62]:
'''
['(G4 not yours AND G4 not yours => -0.0181)',
 '(G4 not yours AND G4 yours AND G4 not yours => -0.1092)',
 '(G4 not yours AND G4 yours AND G4 yours AND H6 not empty => 0.1901)',
 '(G4 not yours AND G4 yours AND G4 yours AND H6 empty => -0.0731)',
 '(G4 yours AND E2 not placed AND C4 not placed AND D7 not placed AND G0 not placed AND E6 not placed AND G2 not placed => 0.0621)',
 '(G4 yours AND E2 not placed AND C4 not placed AND D7 not placed AND G0 not placed AND E6 not placed AND G2 placed => 0.7166)',
 '(G4 yours AND E2 not placed AND C4 not placed AND D7 not placed AND G0 not placed AND E6 placed => 1.4627)',
 '(G4 yours AND E2 not placed AND C4 not placed AND D7 not placed AND G0 placed => 1.1376)',
 '(G4 yours AND E2 not placed AND C4 not placed AND D7 placed => 1.2259)',
 '(G4 yours AND E2 not placed AND C4 placed => 0.6678)',
 '(G4 yours AND E2 placed => 1.0289)']
'''
row_439 = dataset.iloc[488]
row_439[["G4 yours", "D7 placed", "E6 yours", "E6 mine", "E6 empty"]]

G4 yours     9.999998e-01
D7 placed    1.000000e+00
E6 yours     9.479275e-14
E6 mine      6.601343e-20
E6 empty     1.000000e+00
Name: 488, dtype: float32

In [12]:
# compare y_te
print(y_test.shape)
print(y_test_pred.shape)
y_test = y_test.reshape(-1)
print(y_test.shape)
compare = pd.DataFrame({'y_test': y_test, 'y_test_pred': y_test_pred})
compare

NameError: name 'y_test' is not defined